# レッスン 08 LangChainでAgentを作ってみよう

In [5]:
def create_env_file():
    content_1 = "DEEPSEEK_API_KEY=sk-b0a375fa925b445aaaff60ddd5f1b900\n"
    content_2 = "TAVILY_API_KEY=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"

    with open(".env", "w") as f:
        f.write(content_1)
        f.write(content_2)


create_env_file()

import os
from dotenv import load_dotenv

load_dotenv(override=True)

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")

LangChain 1.0における最大の変更点は、間違いなく全く新しいAgent API：`create_agent`の導入です。これにより、LangChain Agentはもはや初期の「少し魔法がかったモデル呼び出し器」ではなく、正式に**コンテキストを認識し、行動能力を持ち、拡張可能でプラガブルなインテリジェントランタイムユニット（Intelligent Runtime Unit）**として定義されるようになりました。簡単に言えば、Agentはもはや「大規模言語モデルで質問に答える」だけではなく、タスクに応じて動的にツールを呼び出し、推論・意思決定を行い、ステップを計画し、外部世界と相互作用する**自律的な実行体**となったのです。

LangChainは1.0版で、すべてのAgentの作成方法を一つのエントリーポイント—`create_agent()`—に統一しました。これは旧版の`create_react_agent`、`create_json_agent`、`create_tool_calling_agent`などの複数の分岐関数を置き換え、開発者が一行のコードであらゆるタイプのエージェントを作成できるようにしました。

LangChain 0.x時代、フレームワーク内のAgentシステムは「断片化」の段階を経験していました。当時の設計思想は**「シナリオに応じて特定のAgentを設計する」**というものでした—思考連鎖推論（ReAct）を実装したければ`create_react_agent`を使い、構造化出力が必要なら`create_structured_chat_agent`を使い、ツール呼び出しには`create_tool_calling_agent`を使うといった具合です。この方式は柔軟でしたが、3つの明らかな問題をもたらしました：
1. **認知的負荷が高い**—各Agentのタイプごとに個別にAPIとパラメータを記憶する必要がある
2. **組み合わせ性が悪い**—複数のAgent間で統一されたスケジューリングができない
3. **エコシステムの断片化**—異なるモジュール間での再利用や協調的な進化が困難

そしてLangChain 1.0に入ると、チームは徹底的なリファクタリングを実施しました：すべてのAgentの作成エントリーポイントを`create_agent()`に統合し、同時に底層では「ミドルウェアメカニズム（Middleware）」と「標準モデルインターフェース（invoke / stream）」を通じてグローバルな統一を実現しました。これによりフレームワークはより軽量で安定し、他のAgentプラットフォームへの統合も容易になりました。

そしてLangChain 1.0のcreate_agent APIの実際の呼び出しプロセスも非常にシンプルです。コードレベルで見ると、モデルとツールを組み合わせるだけでAgentの作成が完了します：


In [2]:
! pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 16.1 MB/s  0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 16.6 MB/s  0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 24.1 MB/s  0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------------ --------------------- 6.0/12.8 MB 28.4 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 30.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 29.7 MB/s  0:00:00

   --- ------------------------------------  2/22 [numpy]
   --- ------------------------------------  2/22 [numpy]
   --- ------------------------------------  2/22 [numpy]
   --- ------------------------------------  2/22 [numpy]
   --- ----------------------------

In [8]:
# 関連ライブラリのインポート
from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek
from langchain_community.tools.tavily_search import TavilySearchResults

# モデルとツールのインポート
web_search = TavilySearchResults(max_results=2)
model = ChatDeepSeek(model="deepseek-chat")

# Agentの作成
agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="あなたはアシスタントで、ツールを呼び出してユーザーの問題解決を支援できます。",
)

# Agentを実行して結果を取得
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "2025年12月の青森地震について詳細を教えてください。",
            }
        ]
    }
)

result["messages"][-1].content

'2025年12月の青森地震について、以下の詳細情報が確認できました：\n\n## 2025年12月8日 青森県東方沖地震の概要\n\n### 基本情報\n- **発生時刻**: 2025年12月8日 23時15分ごろ\n- **震源地**: 青森県東方沖（八戸市の東北東約80km付近）\n- **座標**: 北緯41.0度/東経142.3度\n- **深さ**: 約50-54km\n- **マグニチュード**: M7.5-7.6\n- **最大震度**: 6強（青森県八戸市）\n\n### 震度分布\n- **震度6強**: 青森県八戸市\n- **震度6弱**: 青森県おいらせ町、階上町\n- **震度5強**: 北海道函館市、青森県むつ市、野辺地町、七戸町、東北町、東通村、五戸町、青森南部町、岩手県軽米町、一戸町\n- **震度5弱**: 北海道・青森県・岩手県・宮城県の複数市町村\n- **震度1以上**: 北海道から関東地方、中部地方まで広範囲に観測\n\n### 被害状況（12月10日時点）\n- **人的被害**: 死者0人、負傷者37人（初期報告では負傷者6名）\n- **建物被害**: 住家3件、非住家8件、その他多数の内壁・外壁のひび割れ等\n- **ライフライン**: 一部地域で断水発生（後に解消）\n- **交通**: 道路損傷による通行規制あり（後に解消）\n- **避難者**: 最大2,910人が避難所へ避難\n\n### 津波情報\n- 津波警報・注意報が発表され、岩手県久慈港で0.7mの津波を観測\n- 津波警報・注意報は12月9日朝6時20分にすべて解除\n- 避難所はすべて閉鎖され、避難者は帰宅\n\n### 政府対応\n- **災害救助法適用**: 青森県3市7町2村、岩手県5市4町3村に適用\n- **緊急地震速報**: 発表済み\n- **支援体制**: 各県社会福祉協議会、全社協、NGO団体が支援活動を展開\n\n### 地震の特徴\n- 1968年以来57年ぶりに青森県東方沖で発生したM7以上の地震\n- 2024年1月の能登半島地震（最大震度7）以来の震度6強以上の地震\n- 地震メカニズム: 西北西-東南東方向に圧力軸を持つ逆断層型\n- 太平洋プレートと陸のプレートの境界で発生\n\n### 余震活動\n- 12月

そして底層アーキテクチャの観点から見ると、LangChain 1.0から、公式はフレームワーク全体のアーキテクチャに対して根本的な役割の再定義を行いました：LangGraphは正式にLangChainの底層ランタイムフレームワーク（Runtime Foundation）として確立され、付属的な拡張ではなくなりました。
0.x時代、LangGraphはまだLangChainの「高レベルラッパー」と見なされていました—それは開発者がLangChainを基盤としてさらに複雑なAgent編成、状態遷移、マルチノード制御を実現するためのオプションツールでした。言い換えれば、当時はLangChainがコアで、LangGraphは外付けプラグインでした。
しかし1.0に入ってから、状況は完全に逆転しました：LangGraphはLangChainの底層実行エンジンとして位置づけられ、すべての新しい高レベルAPI（統一されたcreate_agent()、ミドルウェアシステム、状態管理メカニズム、ストリーミングコールバック、イベントスケジューリングなど）は、すべてLangGraphアーキテクチャ上で直接動作するようになりました。
これにより、LangChainはもはや単なる「Prompt組織化と呼び出しライブラリ」ではなく、グラフ構造ベースのインテリジェントエージェントランタイムシステムとなりました：各Agentのライフサイクル、各モデル呼び出し、各ツール実行は、すべてグラフ内のノードまたはエッジの実行プロセスです。LangGraphは安定したスケジューリング、状態の受け渡し、イベントフロー制御を提供し、LangChainは上位層のインターフェースと開発体験を提供します。
つまり、上記Agentの実際の実行ロジックを、LangGraphの言語で表現すると次のようになります：

## create_agent APIのモデル接続方法

`create_agent()`の底層実行メカニズムにおいて、最も重要な3つのコンポーネントは以下の通りです：

1. **モデル（Model）**  
モデルはAgentの「頭脳」であり、タスクの理解と意思決定推論を担当します。静的モデル（固定不変）でも、動的モデル（コンテキストに応じて切り替え）でも構いません。

2. **ツール（Tools）**  
ツールはAgentの「手足」です。モデルは異なるツールを呼び出して特定のサブタスクを実現できます。例えば：
   - `web_search()` → ウェブ検索を実行
   - `calculate()` → 数式を計算
   - `database_query()` → データソースをクエリ

3. **ミドルウェア（Middleware）**  
ミドルウェアは「神経系統」のような存在です。モデル呼び出しの前後でリクエストをインターセプトし、パラメータを修正したり、モデル選択ロジックを動的に調整したりできます。これはLangChain 1.0の強力な拡張メカニズムです。

`create_agent`は2つのモデル使用モードをサポートしています：**静的モデル（Static Model）**と**動的モデル（Dynamic Model）**です。そのうち静的モデルは、Agent作成時に一度設定され、実行プロセス全体で固定されます。この方式はシンプルで安定しており、実験や一般的な本番環境に適しています。例えば：

```python
from langchain.agents import create_agent
agent = create_agent(
    "deepseek:deepseek-chat",
    tools=tools
)
```

この時、LangChainは自動的に「deepseek-chat」を認識し、その完全な識別子「deepseek:deepseek-chat」を推測します。この方式はデモの迅速な構築や標準化されたモデル呼び出しに適しています。

完全な静的モデル文字列マッピングの詳細は以下を参照してください：  

https://reference.langchain.com/python/langchain/models/

また、静的モデルの呼び出しプロセスでは、先ほどの例で示したように、まずモデルをインスタンス化してから、インスタンス化されたモデルを`create_agent`に渡してAgentを作成することもできます。この場合、モデル作成の段階でより多くのパラメータ設定を行うことができます。例えば：

```python
from langchain_deepseek import ChatDeepSeek
from langchain.agents import create_agent

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)

agent = create_agent(model, tools=tools)
```



## create_agentへの内蔵ツール接続とAgent作成フロー

当然ながら、いくつかのツールを接続して初めて、`create_agent`を使用してAgentの作成を完了できます。

ここではまず、最も基本的なLangChainツール接続フロー—LangChain内蔵ツールセットの接続—を紹介します。実は、MCPが爆発的に流行する前から、LangChainエコシステムには既に非常に多くの実用的なツールが内蔵統合されており、開発者はこれらのツールを迅速に呼び出して、より複雑なワークフローの開発を完成させることができます。

LangChain内蔵ツールリスト：https://python.langchain.com/docs/integrations/tools/

その中の代表的なツールは以下の通りです：

| 分野                | ツール名                                               | 機能説明                                         |
| ----------------- | -------------------------------------------------- | -------------------------------------------- |
| **検索エンジンと情報検索**   | 🔹 **DuckDuckGo Search**                           | 無料検索エンジン、ウェブページのタイトル、要約、リンクを返す。オンラインQ&Aに常用。 |
|                   | 🔹 **Brave Search API**                            | プライバシー重視のウェブ検索結果を提供、出力深度とフィルタリングの設定が可能。     |
|                   | 🔹 **Bing Search Tool**                            | Microsoft Bing検索インターフェース、精確な検索と多言語クエリに対応。   |
| **ウェブブラウジングとコンテンツ抽出** | 🔹 **Playwright Browser Toolkit**                  | ブラウザ自動化アクセス、ウェブページスクリーンショット、テキスト抽出機能を提供。   |
|                   | 🔹 **Selenium Toolkit**                            | Playwrightに類似するが、従来のウェブページDOM操作に偏重。         |
| **生産性ツール統合**      | 🔹 **Slack Toolkit**                               | AgentがSlackでメッセージの送信、読取、要約を可能にする。          |
|                   | 🔹 **Jira Toolkit**                                | Jiraプロジェクトチケットのクエリ、作成、更新に使用。               |
|                   | 🔹 **Google Drive / Docs Toolkit**                 | Googleドキュメント、スプレッドシート、ファイル検索を操作。           |
| **データベースとデータ操作**  | 🔹 **SQLDatabase Toolkit**                         | SQLデータベースに接続し、SQLクエリを自動生成・実行。             |
|                   | 🔹 **Faiss / Chroma / Milvus VectorStore Tools**   | ベクトル類似度検索を実行、RAG検索に使用。                     |
| **コードとターミナル実行**   | 🔹 **Python REPL Tool**                            | Pythonコードスニペットを実行し結果を返す（サンドボックス環境）。        |
|                   | 🔹 **Shell Tool (Bash)**                           | Shellコマンドの実行を許可（セキュリティ制限が必要）。              |
|                   | 🔹 **Requests Wrapper Tool**                       | HTTPリクエスト、API呼び出しを発行、外部データとの相互作用に使用。      |
| **マルチモーダルツール**    | 🔹 **DALL-E Tool**                                 | 画像生成、テキスト記述と組み合わせてコンテンツを作成。               |
|                   | 🔹 **SerpAPI Image Search**                        | 画像検索エンジン、画像URLとメタ情報を返す。                    |
|                   | 🔹 **OpenAI Whisper Tool**                         | 音声文字起こしツール、音声ファイルをテキストに変換。                 |
| **知識Q&Aと埋め込み検索**  | 🔹 **VectorStoreRetrieverTool**                    | 埋め込みモデルに基づいて文書断片を検索、RAG Q&Aに使用。          |
|                   | 🔹 **MultiQueryRetrieverTool**                     | 複数のクエリバリエーションを生成して検索カバレッジを向上。             |
| **クラウドとAIサービス**   | 🔹 **OpenAI Functions / Azure Functions Tool**     | AgentがOpenAIまたはAzure定義の関数を呼び出せるようにする。     |
|                   | 🔹 **Anthropic ToolKit**                           | Claudeモデルのツール呼び出し機能を統合。                    |
| **ファイルシステムとデータ処理** | 🔹 **CSV Toolkit**                                 | CSVファイルデータの読み取り、フィルタリング、分析。                |
|                   | 🔹 **Pandas DataFrame Agent Tool**                 | DataFrame上で直接クエリと統計タスクを実行。                 |


ここでは、その中のネットワーク検索ツールを例に、内蔵ツールを`create_agent`に接続する方法を紹介します。

LangChainには`TavilySearchResults`検索ツールが内蔵されており、Tavilyを利用してネットワーク検索と情報クローリングを行うことができます。ここではまず、Tavily公式サイトで登録してAPI-KEYを取得する必要があります（毎月無料枠があります）：https://www.tavily.com/

その後、API-KEYをローカルの`.env`ファイルの`TAVILY_API_KEY`変数に書き込みます。これで呼び出しが可能になります。

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=3)

In [13]:
web_search?

Type:           TavilySearchResults
String form:    max_results=3 api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))
File:           c:\users\wh.yan\miniforge3\envs\lc_env\lib\site-packages\langchain_community\tools\tavily_search\tool.py
Docstring:     
!!! deprecated "0.3.25 Use ``langchain_tavily.TavilySearch`` instead. It will not be removed until langchain-community==1.0."

Tool that queries the Tavily Search API and gets back json.

    Setup:
        Install ``langchain-openai`` and ``tavily-python``, and set environment variable ``TAVILY_API_KEY``.

        .. code-block:: bash

            pip install -U langchain-community tavily-python
            export TAVILY_API_KEY="your-api-key"

    Instantiate:

        .. code-block:: python

            from langchain_community.tools import TavilySearchResults

            tool = TavilySearchResults(
                max_results=5,
                include_answer=True,
                include_raw_content=True,
 

これは高度にカプセル化されたネットワーク検索ツールで、直接呼び出すことができます：

In [14]:
web_search.invoke("2025年12月の青森地震について詳細を教えてください。")

[{'title': '地震情報（2025年12月8日）最大震度6強｜震源地：青森県 ...',
  'url': 'https://typhoon.yahoo.co.jp/weather/jp/earthquake/20251208231519.html',
  'content': '---\n\n現在位置： 天気・災害トップ > 地震情報 > 地震の履歴一覧 > 地震詳細\n\n## 地震情報\n\nLanguage\n\n 日本語\n English\n 繁体中文\n 簡体中文\n 한국어\n\n 全地点の震度\n 各地域の震度\n\n 1震度1\n 2震度2\n 3震度3\n 4震度4\n 5-震度5弱\n 5+震度5強\n 6-震度6弱\n 6+震度6強\n 7震度7\n 震央震央\n\n 1震度1\n 2震度2\n 3震度3\n 4震度4\n 5-震度5弱\n 5+震度5強\n 6-震度6弱\n 6+震度6強\n 7震度7\n 震央震央\n\n|  |  |\n --- |\n| 発生時刻 | 2025年12月8日 23時15分ごろ |\n| 震源地 | 青森県東方沖（八戸の東北東80km付近） |\n| 最大震度 | 6強 |\n| マグニチュード | 7.6 |\n| 深さ | 50km |\n| 緯度/経度 | 北緯41.0度/東経142.3度 |\n| 情報 | 津波警報等（大津波警報・津波警報あるいは津波注意報）を発表中です。この地震について、緊急地震速報を発表しています。 | [...] |  |  |  |  |\n ---  --- |\n| 震度6強 | |  |  |  --- | | 青森県 | 八戸市 | |\n| 震度6弱 | |  |  |  --- | | 青森県 | おいらせ町\u3000 階上町 | |\n| 震度5強 | |  |  |  --- | | 北海道 | 函館市 | | 青森県 | むつ市\u3000 野辺地町\u3000 七戸町\u3000 東北町\u3000 東通村\u3000 五戸町\u3000 青森南部町 | | 岩手県 | 軽米町\u3000 一戸町 | |\n| 震度5弱 | |  |  |  --- | | 北海道 | 苫小牧市\u3000 千歳市\u3000 新篠津村\u3000 南幌町\

また、外部ツールとしてcreate_agentに直接渡すこともできます。

In [ ]:
# 関連ライブラリのインポート
from langchain.agents import create_agent
from langchain_deepseek import ChatDeepSeek
from langchain_community.tools.tavily_search import TavilySearchResults

# モデルとツールのインポート
web_search = TavilySearchResults(max_results=2)
model = ChatDeepSeek(model="deepseek-chat")

# Agentの作成
agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="あなたはアシスタントで、ツールを呼び出してユーザーの問題解決を支援できます。",
)

# Agentを実行して結果を取得
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "2025年12月の青森地震について詳細を教えてください。",
            }
        ]
    }
)

result["messages"][-1].content

そして、今回の実行プロセスを注意深く観察すると、今回のツール呼び出しは依然として典型的なFunction calling実行フローであり、ユーザーが最初にメッセージを発信するものを含めて、合計７つのメッセージが作成されました。

In [15]:
result["messages"]

[HumanMessage(content='2025年12月の青森地震について詳細を教えてください。', additional_kwargs={}, response_metadata={}, id='08a7a7b0-621b-4f69-a4b7-5b4c004a3dcf'),
 AIMessage(content='2025年12月の青森地震について調べます。現在の情報をお探しですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 376, 'total_tokens': 461, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 320}, 'prompt_cache_hit_tokens': 320, 'prompt_cache_miss_tokens': 56}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '8f249bc8-44f0-4a83-8a70-35ff4bda0d6e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--fe32af47-5eef-4df6-a661-18e04288f103-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025年12月 青森地震 発生 詳細 震度 被害'}, 'id': 'call_00_9fN9lsoszvYHM999kc7Dg5gk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 376, 'output_tokens': 8

In [16]:
result["messages"][-3]

AIMessage(content='さらに被害状況や対応について詳しく調べます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 4244, 'total_tokens': 4326, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 2048}, 'prompt_cache_hit_tokens': 2048, 'prompt_cache_miss_tokens': 2196}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '115dd119-5f56-4d6d-b96f-d8886530fd04', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--41a0e4b7-e766-47db-bdbd-3635e784e3c9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025年12月8日 青森地震 被害 死者 負傷者 津波 避難'}, 'id': 'call_00_RfdtEYIlwZ0UwNk2OJ1QGlPW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 4244, 'output_tokens': 82, 'total_tokens': 4326, 'input_token_details': {'cache_read': 2048}, 'output_token_details': {}})

In [17]:
result["messages"][-5]

AIMessage(content='さらに詳細な情報を探してみます。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 2023, 'total_tokens': 2101, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 448}, 'prompt_cache_hit_tokens': 448, 'prompt_cache_miss_tokens': 1575}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '9c46cddb-aacc-4b66-b145-779e4564559d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--09e00261-eb5b-424c-8646-df3d52110f97-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2025年12月8日 青森県東方沖 地震 最大震度6強 被害状況'}, 'id': 'call_00_yV2PjNhz8DyoqBihCqTMUYzh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2023, 'output_tokens': 78, 'total_tokens': 2101, 'input_token_details': {'cache_read': 448}, 'output_token_details': {}})

In [21]:
result["messages"][-1].content

'2025年12月の青森地震について、以下の詳細情報が確認できました：\n\n## 2025年12月8日 青森県東方沖地震の概要\n\n### 基本情報\n- **発生時刻**: 2025年12月8日 23時15分ごろ\n- **震源地**: 青森県東方沖（八戸市の東北東約80km付近）\n- **座標**: 北緯41.0度/東経142.3度\n- **深さ**: 約50-54km\n- **マグニチュード**: M7.5-7.6\n- **最大震度**: 6強（青森県八戸市）\n\n### 震度分布\n- **震度6強**: 青森県八戸市\n- **震度6弱**: 青森県おいらせ町、階上町\n- **震度5強**: 北海道函館市、青森県むつ市、野辺地町、七戸町、東北町、東通村、五戸町、青森南部町、岩手県軽米町、一戸町\n- **震度5弱**: 北海道・青森県・岩手県・宮城県の複数市町村\n- **震度1以上**: 北海道から関東地方、中部地方まで広範囲に観測\n\n### 被害状況（12月10日時点）\n- **人的被害**: 死者0人、負傷者37人（初期報告では負傷者6名）\n- **建物被害**: 住家3件、非住家8件、その他多数の内壁・外壁のひび割れ等\n- **ライフライン**: 一部地域で断水発生（後に解消）\n- **交通**: 道路損傷による通行規制あり（後に解消）\n- **避難者**: 最大2,910人が避難所へ避難\n\n### 津波情報\n- 津波警報・注意報が発表され、岩手県久慈港で0.7mの津波を観測\n- 津波警報・注意報は12月9日朝6時20分にすべて解除\n- 避難所はすべて閉鎖され、避難者は帰宅\n\n### 政府対応\n- **災害救助法適用**: 青森県3市7町2村、岩手県5市4町3村に適用\n- **緊急地震速報**: 発表済み\n- **支援体制**: 各県社会福祉協議会、全社協、NGO団体が支援活動を展開\n\n### 地震の特徴\n- 1968年以来57年ぶりに青森県東方沖で発生したM7以上の地震\n- 2024年1月の能登半島地震（最大震度7）以来の震度6強以上の地震\n- 地震メカニズム: 西北西-東南東方向に圧力軸を持つ逆断層型\n- 太平洋プレートと陸のプレートの境界で発生\n\n### 余震活動\n- 12月